In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import date, timedelta
from pandas.tseries.offsets import * 
import calendar

pd.options.display.max_rows = 30

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
#yesterday = today - timedelta(days=1)
today

datetime.date(2024, 9, 11)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2024-09-11
Today in mm-dd format: 09-11
yesterday: 2024-09-10 00:00:00


In [3]:
yesterday = yesterday.date()
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2024-08-13 00:00:00   Tuesday


### Dividend

In [4]:
sql = """SELECT * from dividend"""
df = pd.read_sql(sql, const)
df

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,WHART,0.130,0.259,0.1930,0.1915,0.7735,0.0,0.0,30000.0,2024-08-20,2024-09-06,,1
1,CPNREIT,0.257,0.257,0.0000,0.3618,0.8758,0.0,0.0,60000.0,2024-08-26,2024-09-11,,1
2,MCS,0.320,0.000,0.2000,0.0000,0.5200,0.0,0.0,78000.0,2024-08-23,2024-09-10,,1
3,AIMIRT,0.215,0.223,0.1433,0.2867,0.8680,0.0,0.0,12500.0,2024-08-26,2024-09-12,,1
4,SCC,3.500,0.000,2.5000,0.0000,6.0000,0.0,0.0,600.0,2024-08-07,2024-08-23,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,TOA,0.350,0.000,0.3300,0.0000,0.6800,0.0,0.0,1000.0,2024-08-28,2024-09-13,,1
27,RCL,0.500,0.000,0.5000,0.0000,1.0000,0.0,0.0,27000.0,2024-08-22,2024-09-06,,1
28,PTG,0.350,0.000,0.0000,0.0000,0.3500,0.0,0.0,3600.0,2024-03-06,2024-05-17,,0
29,AWC,0.050,0.000,0.0000,0.0000,0.0500,0.0,0.0,9000.0,2024-05-09,2024-05-24,,0


In [5]:
sqlUpd = """
UPDATE dividend
SET actual = "0"
WHERE name NOT IN ('CPNREIT','DIF','JASIF')"""
print(sqlUpd)


UPDATE dividend
SET actual = "0"
WHERE name NOT IN ('CPNREIT','DIF','JASIF')


In [6]:
rp = const.execute(sqlUpd)
rp.rowcount

28

### Solving problem

In [7]:
name = 'JMART'
sql = """
SELECT * 
FROM buy
WHERE name = '%s'
"""
sql = sql % name
df = pd.read_sql(sql, const)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,JMART,2023-01-20,6800.0,33.0,0.0,0.0,1,0.0,3,A2


In [8]:
df_tmp = df.copy()
df_tmp

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,JMART,2023-01-20,6800.0,33.0,0.0,0.0,1,0.0,3,A2


In [9]:
sqlDel = '''
DELETE
FROM buy
WHERE name = "AIMIRT"'''
rp = const.execute(sqlDel)
rp.rowcount

1

In [41]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [42]:
rcd = df_tmp.values.tolist()
rcd

[['JMART',
  datetime.date(2023, 1, 20),
  6800.0,
  33.0,
  0.0,
  0.0,
  1,
  0.0,
  '3',
  'A2']]

In [43]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

### Change grade & period

In [14]:
name = 'WHART'

sql = """
SELECT * 
FROM buy
WHERE name = '%s'"""
sql = sql % name
df = pd.read_sql(sql, const)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,WHART,2022-01-05,20000.0,12.3,0.0,0.0,1,0.7735,4,C1


In [15]:
new_grade = 'C1'
new_period = '2'
sqlUpd = """
UPDATE buy
SET grade = "%s", period = "%s" WHERE name = "%s"
"""
sqlUpd = sqlUpd % (new_grade, new_period, name)
print(sqlUpd)
rp = const.execute(sqlUpd)
rp.rowcount


UPDATE buy
SET grade = "C1", period = "2" WHERE name = "WHART"



1

In [16]:
sql = """
SELECT * 
FROM buy
WHERE name = '%s'"""
sql = sql % name
df = pd.read_sql(sql, const)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,WHART,2022-01-05,20000.0,12.3,0.0,0.0,1,0.7735,2,C1


In [17]:
sql = '''
SELECT *
FROM buy ORDER BY period, name
'''
buy = pd.read_sql(sql, const)
buy['costAmt'] = buy.volbuy * buy.price
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy.sort_values(['period','name'],ascending=[True,True])

,name,date,shares,price,active,dividend,period,grade,costAmt
0,AH,2023-06-08,1200,37.00,1,1.2500,1,C3,44400.0
1,IVL,2022-08-29,8000,38.00,1,0.7750,1,C1,304000.0
2,ORI,2022-04-07,45000,11.00,1,0.3000,1,C3,495000.0
3,RCL,2021-11-08,27000,38.75,1,1.0000,1,C2,1046250.0
4,SCC,2021-09-20,600,405.00,1,6.0000,1,C2,243000.0
5,STA,2021-06-15,5000,40.00,1,1.0000,1,C2,200000.0
6,AIMIRT,2023-08-17,12500,10.90,1,0.8680,2,B2,136250.0
7,CPNREIT,2022-08-16,55000,18.00,1,0.8758,2,B3,990000.0
8,DIF,2020-08-01,30000,14.70,1,0.8972,2,B3,441000.0
9,GVREIT,2022-08-24,66000,7.75,1,0.7833,2,C1,511500.0


In [17]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'costAmt':'{:,.2f}',    
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}',    
}

In [18]:
buy.style.format(format_dict)

,name,date,shares,price,active,dividend,period,grade,costAmt
0,AH,2023-06-08,"1,200",37.00,1,1.2500,1,C3,"44,400.00"
1,IVL,2022-08-29,"8,000",38.00,1,0.7750,1,C1,"304,000.00"
2,ORI,2022-04-07,"48,000",10.60,1,0.4600,1,C3,"508,800.00"
3,RCL,2021-11-08,"27,000",38.75,1,1.0000,1,C2,"1,046,250.00"
4,SCC,2021-09-20,600,405.00,1,6.0000,1,C2,"243,000.00"
5,STA,2021-06-15,"5,000",40.00,1,1.0000,1,C2,"200,000.00"
6,ASP,2022-03-07,"30,000",3.80,1,0.1700,2,B4,"114,000.00"
7,CPNREIT,2022-08-16,"60,000",17.55,1,0.8758,2,B3,"1,053,000.00"
8,DIF,2020-08-01,"30,000",14.70,1,0.8972,2,B3,"441,000.00"
9,GVREIT,2022-08-24,"66,000",7.75,1,0.7833,2,C1,"511,500.00"


In [19]:
sql = '''
SELECT name, year, quarter, q_eps, aq_eps, publish_date
FROM epss
WHERE year = 2022 AND quarter = 2'''
epss = pd.read_sql(sql, conlt)
epss.sample(5)

,name,year,quarter,q_eps,aq_eps,publish_date
177,RATCH,2022,2,1.2000,2.0600,2022-08-15
176,MCS,2022,2,-0.1500,0.4100,2022-08-15
80,EASTW,2022,2,0.1500,0.3000,2022-08-10
148,POPF,2022,2,0.1858,0.3863,2022-08-11
99,WHAUP,2022,2,0.0500,0.0700,2022-08-10


In [20]:
df_merge = pd.merge(buy, epss, on='name', how='inner')
df_merge.sort_values(['period','name'],ascending=[True,True])

,name,date,shares,price,active,dividend,period,grade,costAmt,year,quarter,q_eps,aq_eps,publish_date
0,AH,2023-06-08,1200,37.00,1,1.2500,1,C3,44400.0,2022,2,1.1500,2.2900,2022-08-15
1,IVL,2022-08-29,8000,38.00,1,0.7750,1,C1,304000.0,2022,2,3.5800,6.0500,2022-08-10
2,ORI,2022-04-07,48000,10.60,1,0.4600,1,C3,508800.0,2022,2,0.4708,0.7716,2022-08-15
3,RCL,2021-11-08,27000,38.75,1,1.0000,1,C2,1046250.0,2022,2,8.8790,18.8000,2022-08-05
4,SCC,2021-09-20,600,405.00,1,6.0000,1,C2,243000.0,2022,2,8.2800,15.6500,2022-07-27
5,STA,2021-06-15,5000,40.00,1,1.0000,1,C2,200000.0,2022,2,0.7400,1.7300,2022-08-10
6,ASP,2022-03-07,30000,3.80,1,0.1700,2,B4,114000.0,2022,2,0.0200,0.0900,2022-07-27
7,CPNREIT,2022-08-16,60000,17.55,1,0.8758,2,B3,1053000.0,2022,2,0.3334,0.5147,2022-08-08
8,DIF,2020-08-01,30000,14.70,1,0.8972,2,B3,441000.0,2022,2,0.2706,0.5429,2022-08-11
9,GVREIT,2022-08-24,66000,7.75,1,0.7833,2,C1,511500.0,2022,2,0.2342,0.4748,2022-05-13


In [21]:
sql = '''
SELECT name, market
FROM stocks
ORDER BY name'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(246, 2)

In [22]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))    
]
values = ['SET50','SET100','mai']

In [23]:
my_stocks["mrkt"] = np.select(filters, values, default='SET')
my_stocks.head()

,name,market,mrkt
0,3BBIF,SET,SET
1,ACE,SET100 / SETTHSI,SET100
2,ADVANC,SET50 / SETHD / SETTHSI,SET50
3,AEONTS,SET,SET
4,AH,sSET / SETTHSI,SET


In [24]:
df_merge2 = pd.merge(df_merge,my_stocks,on='name',how='inner')
df_merge2

,name,date,shares,price,active,dividend,period,grade,costAmt,year,quarter,q_eps,aq_eps,publish_date,market,mrkt
0,AH,2023-06-08,1200,37.00,1,1.2500,1,C3,44400.0,2022,2,1.1500,2.2900,2022-08-15,sSET / SETTHSI,SET
1,IVL,2022-08-29,8000,38.00,1,0.7750,1,C1,304000.0,2022,2,3.5800,6.0500,2022-08-10,SET50 / SETTHSI,SET50
2,ORI,2022-04-07,48000,10.60,1,0.4600,1,C3,508800.0,2022,2,0.4708,0.7716,2022-08-15,SET100 / SETHD / SETTHSI,SET100
3,RCL,2021-11-08,27000,38.75,1,1.0000,1,C2,1046250.0,2022,2,8.8790,18.8000,2022-08-05,SET100 / SETCLMV / SETWB,SET100
4,SCC,2021-09-20,600,405.00,1,6.0000,1,C2,243000.0,2022,2,8.2800,15.6500,2022-07-27,SET50 / SETCLMV / SETHD / SETTHSI,SET50
5,STA,2021-06-15,5000,40.00,1,1.0000,1,C2,200000.0,2022,2,0.7400,1.7300,2022-08-10,SET100 / SETTHSI / SETWB,SET100
6,ASP,2022-03-07,30000,3.80,1,0.1700,2,B4,114000.0,2022,2,0.0200,0.0900,2022-07-27,sSET,SET
7,CPNREIT,2022-08-16,60000,17.55,1,0.8758,2,B3,1053000.0,2022,2,0.3334,0.5147,2022-08-08,SET,SET
8,DIF,2020-08-01,30000,14.70,1,0.8972,2,B3,441000.0,2022,2,0.2706,0.5429,2022-08-11,SET,SET
9,GVREIT,2022-08-24,66000,7.75,1,0.7833,2,C1,511500.0,2022,2,0.2342,0.4748,2022-05-13,SET,SET


In [25]:
cols = 'period grade mrkt name shares price costAmt '.split()
df_merge2['shares'] = df_merge['shares'].astype(int)
df_merge2[cols].sort_values(['period', 'name'],ascending=[True,True]).style.format(format_dict)

,period,grade,mrkt,name,shares,price,costAmt
0,1,C3,SET,AH,"1,200",37.00,"44,400.00"
1,1,C1,SET50,IVL,"8,000",38.00,"304,000.00"
2,1,C3,SET100,ORI,"48,000",10.60,"508,800.00"
3,1,C2,SET100,RCL,"27,000",38.75,"1,046,250.00"
4,1,C2,SET50,SCC,600,405.00,"243,000.00"
5,1,C2,SET100,STA,"5,000",40.00,"200,000.00"
6,2,B4,SET,ASP,"30,000",3.80,"114,000.00"
7,2,B3,SET,CPNREIT,"60,000",17.55,"1,053,000.00"
8,2,B3,SET,DIF,"30,000",14.70,"441,000.00"
9,2,C1,SET,GVREIT,"66,000",7.75,"511,500.00"


In [26]:
df_merge2.groupby('period')['costAmt'].sum()

period
1    2346450.0
2    4827600.0
3    2667980.0
4    1859300.0
Name: costAmt, dtype: float64

### End of Quarterly Process

In [27]:
# look again at some of the data
catvars = ['period','grade']

for col in catvars:
  print(col, df_merge2[col].value_counts().sort_index(), sep="\n\n", end="\n\n\n")

period

1     6
2    10
3    10
4     4
Name: period, dtype: int64


grade

A1    2
A2    4
A4    4
B1    1
B2    2
B3    2
B4    2
C1    4
C2    5
C3    3
C4    1
Name: grade, dtype: int64




In [28]:
df_merge2.loc[
    df_merge2.grade == 'A2',
['period','name','grade','costAmt']].sort_values(['period','name'],ascending=[True,True]).style.format(format_dict)

,period,name,grade,costAmt
17,3,JMART,A2,"224,400.00"
18,3,JMART,A2,"224,400.00"
19,3,KCE,A2,"772,500.00"
22,3,SINGER,A2,"148,800.00"


In [29]:
df_merge2.groupby('period').agg(
   {'costAmt':'sum','shares':'sum'}
).round(
   {'costAmt':2, 'shares':0}
).style.format(format_dict)

,costAmt,shares
period,,
1,"2,346,450.00","89,800"
2,"4,827,600.00","473,500"
3,"2,667,980.00","100,200"
4,"1,859,300.00","168,500"


In [30]:
df_merge2.replace(
   {
      'period':{'1':'Disposal','2':'Hi-Div','3':'Long Term','4':'Short Term'}
   }
).sample(5)

,name,date,shares,price,active,dividend,period,grade,costAmt,year,quarter,q_eps,aq_eps,publish_date,market,mrkt
16,AWC,2023-06-15,9000,4.96,1,0.05,Long Term,B1,44640.0,2022,2,0.0243,0.0444,2022-08-10,SET50 / SETTHSI,SET50
23,SYNEX,2022-01-07,15000,28.70,1,0.36,Long Term,B2,430500.0,2022,2,0.2200,0.4800,2022-08-08,sSET / SETTHSI,SET
6,ASP,2022-03-07,30000,3.80,1,0.17,Hi-Div,B4,114000.0,2022,2,0.0200,0.0900,2022-07-27,sSET,SET
25,TOA,2023-09-06,1000,26.00,1,0.68,Long Term,A1,26000.0,2022,2,0.1800,0.3800,2022-08-15,SETTHSI,SET
26,3BBIF,2018-05-17,130000,10.00,1,0.16,Short Term,C2,1300000.0,2022,2,0.1765,0.3783,2022-08-09,SET,SET


In [31]:
cols = 'period grade name shares price costAmt dividend q_eps aq_eps mrkt'.split()
df_merge2.replace(
   {
      'period':{'1':'Disposal','2':'Hi-Div','3':'Long Term','4':'Short Term'}
   }
)[cols].sort_values(['period','name'],ascending=[True,True]).style.format(format_dict)

,period,grade,name,shares,price,costAmt,dividend,q_eps,aq_eps,mrkt
0,Disposal,C3,AH,"1,200",37.00,"44,400.00",1.2500,1.150000,2.290000,SET
1,Disposal,C1,IVL,"8,000",38.00,"304,000.00",0.7750,3.580000,6.050000,SET50
2,Disposal,C3,ORI,"48,000",10.60,"508,800.00",0.4600,0.470800,0.771600,SET100
3,Disposal,C2,RCL,"27,000",38.75,"1,046,250.00",1.0000,8.879000,18.800000,SET100
4,Disposal,C2,SCC,600,405.00,"243,000.00",6.0000,8.280000,15.650000,SET50
5,Disposal,C2,STA,"5,000",40.00,"200,000.00",1.0000,0.740000,1.730000,SET100
6,Hi-Div,B4,ASP,"30,000",3.80,"114,000.00",0.1700,0.020000,0.090000,SET
7,Hi-Div,B3,CPNREIT,"60,000",17.55,"1,053,000.00",0.8758,0.333400,0.514700,SET
8,Hi-Div,B3,DIF,"30,000",14.70,"441,000.00",0.8972,0.270600,0.542900,SET
9,Hi-Div,C1,GVREIT,"66,000",7.75,"511,500.00",0.7833,0.234200,0.474800,SET


### Start of change grade after quarterly eps

In [39]:
name = 'BCH'
sql = '''
SELECT * FROM buy
WHERE name = "%s"
'''
sql = sql % name
df = pd.read_sql(sql, const)
df

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,BCH,2021-09-07,4000.0,21.7,0.0,0.0,1,0.37,4,C1


In [40]:
sql = '''
UPDATE buy
SET period = "4", grade = "C2"
WHERE name = "%s"
'''
sql = sql % name
rp = const.execute(sql)
rp.rowcount

1

### End of change grade after quarterly eps

### Change period by name

In [34]:
name = 'MAKRO'
period = '3'
sqlUpd = '''
UPDATE buy
SET period = "%s"
WHERE name =  "%s"'''
sqlUpd = sqlUpd % (period, name)
print(sqlUpd)


UPDATE buy
SET period = "3"
WHERE name =  "MAKRO"


In [35]:
rp = const.execute(sqlUpd)
rp.rowcount

0

### End of Change period according to grade

### Change grade of specific stock

In [36]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ('ANAN','LH','MCS','ORI')"""
print(sql)


UPDATE buy
SET period = "2"
WHERE name IN ('ANAN','LH','MCS','ORI')


In [37]:
sql = """
UPDATE buy
SET period = "3"
WHERE name IN ('AMATA')"""
print(sql)


UPDATE buy
SET period = "3"
WHERE name IN ('AMATA')


In [38]:
rp = con.execute(sql)
rp.rowcount

NameError: name 'con' is not defined

### End of Change grade of specific stock

In [ ]:
sql = """
INSERT INTO buy
VALUES('GC','2018-02-28',10000,5.3,)"""

In [ ]:
sql = '''
UPDATE buy
SET period = "4"
WHERE grade <>  "A4"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = '''
UPDATE buy
SET period = "1"
WHERE substr(grade,1,1) <>  "A"'''
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Individual record change

In [ ]:
sql = """
UPDATE buy
SET period = "2"
WHERE name IN ('GC','LH','TISCO','MCS','BR')"""
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Par value change

In [ ]:
sql = """
SELECT *
FROM price
LIMIT 1"""
prices = pd.read_sql(sql, const)
prices.dtypes

In [ ]:
name = "DELTA"
effective_date = "2023-04-27"

In [ ]:
sql = """
UPDATE price
SET price = price/10, maxp = maxp/10, minp = minp/10, opnp = opnp/10,
qty = qty*10
WHERE name = '%s' AND date <= '%s'
"""
sql = sql % (name, effective_date)
print(sql)

In [ ]:
rp = const.execute(sql)
rp.rowcount

In [ ]:
sql = """
SELECT *
FROM buy
LIMIT 1"""
buys = pd.read_sql(sql, con)
buys.dtypes

In [ ]:
sql = """
UPDATE buy
SET volbuy = 4000, price = 50.8
WHERE name = '%s' AND active = 1
"""
sql = sql % (name)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

### Name change

In [ ]:
old_name = "EPCO"
new_name = "EP"

In [ ]:
sql = """
SELECT *
FROM stockname
WHERE name = '%s'
"""
sql = sql % (old_name)
stock = pd.read_sql(sql, con)
stock

In [ ]:
sql = """
UPDATE stockname
SET name = '%s'
WHERE name = '%s'
"""
sql = sql % (new_name, old_name)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = """
SELECT *
FROM price
WHERE name = '%s'
"""
sql = sql % (old_name)
price = pd.read_sql(sql, con)
price.shape

In [ ]:
sql = """
UPDATE price
SET name = '%s'
WHERE name = '%s'
"""
sql = sql % (new_name, old_name)
print(sql)

In [ ]:
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = '''
SELECT * FROM price WHERE name = 'ASK' ORDER BY date DESC'''
price = pd.read_sql(sql, con)
price

In [ ]:
sql = '''
DELETE FROM price WHERE date = "2021-12-14"'''
rp = con.execute(sql)
rp.rowcount

In [ ]:
sql = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sql)

In [ ]:
rcd = ['TSE', '2022-01-21', 40000.0, 2.60, 0, 0, 1, 0.10, '3', 'A4']
rcd

In [ ]:
rp = con.execute(sql, rcd)
rp.rowcount

In [ ]:
sqlDel = '''
DELETE
FROM buy
WHERE name = "TSE"'''
rp = con.execute(sqlDel)
rp.rowcount

### SETIndex table

In [ ]:
sql = '''
SELECT * FROM setindex WHERE setindex IS Null'''
df = pd.read_sql(sql, con)
df

In [ ]:
sqlUpd = """
UPDATE setindex
SET setindex = 1661.89 WHERE setindex IS Null"""
print(sqlUpd)

In [ ]:
rp = con.execute(sqlUpd)
rp.rowcount

In [ ]:
sqlIns = """
INSERT INTO setindex
VALUES('2022-04-25',1675.33)"""
print(sqlIns)

In [ ]:
rp = con.execute(sqlIns)
rp.rowcount

In [ ]:
name = 'KCE'
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE name = '%s' 
ORDER BY date DESC"""
sql = sql % (name)
print(sql)

In [ ]:
df_temp = pd.read_sql(sql, const)
df_temp

### Delete prices by date

In [ ]:
today = '2024-06-18'

In [ ]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date = '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)

In [ ]:
sql = """
SELECT *
FROM price 
WHERE date = '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)

In [ ]:
df_price = pd.read_sql(sql, const)
df_price.shape

In [ ]:
df_price.head()

In [ ]:
sqlUpd = "UPDATE price SET date = '2023-12-08' WHERE date = '2023-12-11'"
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sqlDel = """
DELETE
FROM price 
WHERE date = '%s' 
"""
sqlDel = sqlDel % (today)
#sqlDel = sqlDel % (yesterday)
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

In [ ]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (yesterday)
print(sql)

In [ ]:
df_price_yesterday = pd.read_sql(sql, const)
df_price_yesterday.shape

In [ ]:
sql = """
SELECT *
FROM price 
WHERE date >= '%s' 
AND name = 'MCS'
ORDER BY name, date"""
sql = sql % (yesterday)
print(sql)

In [ ]:
df_price = pd.read_sql(sql, const)
df_price

In [ ]:
sqlIns = """
INSERT INTO price
VALUES('SINGER','2023-11-16',10.3,10.7,10.1,17108500,10.3)"""
print(sqlIns)

In [ ]:
rp = const.execute(sqlIns)
rp.rowcount

### Table stockname

In [ ]:
sql = """
SELECT *
FROM stockname
WHERE name = 'CPNRF'
"""
print(sql)

In [ ]:
df_temp = pd.read_sql(sql, const)
df_temp

In [ ]:
sqlUpd = """
UPDATE stockname
SET name = 'CPNREIT'
WHERE name = 'CPNRF'
"""
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT *
FROM price
ORDER BY date DESC, name ASC
LIMIT 10"""
prices = pd.read_sql(sql, const)
prices

In [ ]:
file_name = 'Price-Sample.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name
one_file = one_path + file_name

prices.sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
prices.sort_values(['name'],ascending=[True]).to_csv(data_file, index=False)
prices.sort_values(['name'],ascending=[True]).to_csv(osd_file, index=False)
prices.sort_values(['name'],ascending=[True]).to_csv(one_file, index=False)